# Tutorial: **δHBV 1.1p**

---

This notebook demonstrates training and forward simulation with the δHBV 1.1p model developed by [Yalan Song et al. (2025)](https://doi.org/10.22541/essoar.172304428.82707157/v2). A pre-trained model is provided for those who only wish to run the model forward.

For explanation of model structure, methodologies, data, and performance metrics, please refer to Song's publication [below](#publication). If you find this code is useful in your own work, please include the aforementioned citation.

**Note**: If you are new to the δMG framework, we suggest first looking at our [δHBV 1.0 tutorial](./../hydrology/example_dhbv.ipynb).

<br>

### Before Running:
- **Environment**: See [setup.md](./../../docs/setup.md) for ENV setup. δMG must be installed with dependencies + [hydrodl2](https://github.com/mhpi/hydrodl2) to run this notebook.

- **Model**: Download pretrained δHBV 1.1p model weights from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/models/3-dhbv_1_1p.zip). Then update the model config:

    - In [config_dhbv_1_1p.yaml](./../conf/config_dhbv.yaml), update `model_dir` with the path to your directory containing both trained model weights `hbv_1_1p_ep50.pt` (or *ep100*) **and** `normalization_statistics.json`.
    - **Note**: make sure this path includes the last closing forward slash: e.g., `./your/path/to/model/`.

- **Data**: Download our CAMELS ([details here](https://ral.ucar.edu/solutions/products/camels)) data extraction from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/data/1-camels.zip). Then, update the data configs:

    - In [camels_531.yaml](./../conf/observations/camels_531.yaml) and [camels_671.yaml](./../conf/observations/camels_671.yaml), update...
        1. `data_path` with path to `camels_daymetv2`,
        2. `gage_info` with path to `gage_id.npy`,
        3. `subset_path` with path to `531sub_id.txt` (camels_531 only).

    - The full 671-basin or 531-basin CAMELS datasets can be selected by setting `observations: camels_671` or `camels_531` in the model config, respectively.

- **Hardware**: The NNs used in this model require CUDA support only available with Nvidia GPUs. For those without access, T4 GPUs can be used when running this notebook with δMG on [Google Colab](https://colab.research.google.com/).

<br>

### Publication:

*Yalan Song, Kamlesh Sawadekar, Jonathan M Frame, Ming Pan, Martyn Clark, Wouter J M Knoben, Andrew W Wood, Trupesh Patel, Chaopeng Shen. "Physics-informed, Differentiable Hydrologic  Models for Capturing Unseen Extreme Events." ESS Open Archive (2025). https://doi.org/10.22541/essoar.172304428.82707157/v2.*

<br>

### Issues:
For questions, concerns, bugs, etc., please reach out by posting an [issue](https://github.com/mhpi/generic_deltamodel/issues).

---


<br>

## 1. Forward δHBV 1.1p

After completing [these](#before-running) steps, forward δHBV 1.1p with the code block below.

Note:
- The settings defined in the config [config_dhbv_1_1p.yaml](./../conf/config_dhbv_1_1p.yaml) are set to replicate benchmark performance on 531 CAMELS basins.
- The first year (`warm_up` in the config, default is 365 days) of the inference period is used for initializing HBV's internal states (water storages) and is, therefore, excluded from the model's prediction output.

### 1.1 Demonstration

In [ ]:
import sys

sys.path.append('../../')

from example import load_config

from dmg import ModelHandler
from dmg.core.utils import import_data_loader, print_config, set_randomseed

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#


# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'sim'
print_config(config)

# Set random seed for reproducibility.
set_randomseed(config['seed'])

# 2. Initialize the differentiable dHBV 1.1p model (LSTM + HBV 1.1p).
model = ModelHandler(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=True, overwrite=False)

# 4. Forward the model to get the predictions.
output = model(
    data_loader.eval_dataset,
    eval=True,
)

print("-------------\n")
print(
    f"Streamflow predictions (mm/day) for {output['Hbv_1_1p']['streamflow'].shape[0]} days and "
    f"{output['Hbv_1_1p']['streamflow'].shape[1]} basins ~ \nShowing the first 5 days for "
    f"first basin: \n {output['Hbv_1_1p']['streamflow'][:5, :1].cpu().detach().numpy().squeeze()}",
)

### 1.2 Visualizing Model Predictions

After running model inference we can, e.g., view the hydrograph for one of the basins to see we are getting expected outputs.

We can do this with our target variable, streamflow, for instance (though, there are many other states and fluxes we can view -- see cell output below).

In [ ]:
import numpy as np

from dmg.core.data import txt_to_array
from dmg.core.post import plot_hydrograph
from dmg.core.utils import Dates

# ------------------------------------------#
# Choose a basin by USGS gage ID to plot.
GAGE_ID = 1022500
TARGET = 'streamflow'

# Resample to 3-day prediction. Options: 'D', 'W', 'M', 'Y'.
RESAMPLE = '3D'

# Set the paths to the gage ID lists...
GAGE_ID_PATH = config['observations']['gage_info']  # ./gage_id.npy
GAGE_ID_531_PATH = config['observations']['subset_path']  # ./531sub_id.txt
# ------------------------------------------#


# 1. Get the streamflow predictions and daily timesteps of the prediction window.
print(f"HBV states and fluxes: {list(output['Hbv_1_1p'].keys())} \n")

pred = output['Hbv_1_1p'][TARGET]
timesteps = Dates(config['sim'], config['model']['rho']).batch_daily_time_range

# Remove warm-up period to match model output (see Note above.)
timesteps = timesteps[config['model']['warm_up'] :]


# 2. Load the gage ID lists and get the basin index.
gage_ids = np.load(GAGE_ID_PATH, allow_pickle=True)
gage_ids_531 = txt_to_array(GAGE_ID_531_PATH)

print(f"First 20 available gage IDs: \n {gage_ids[:20]} \n")
print(f"First 20 available gage IDs (531 subset): \n {gage_ids_531[:20]} \n")

if config['observations']['name'] == 'camels_671':
    if GAGE_ID in gage_ids:
        basin_idx = list(gage_ids).index(GAGE_ID)
    else:
        raise ValueError(
            f"Basin with gage ID {GAGE_ID} not found in the CAMELS 671 dataset.",
        )

elif config['observations']['name'] == 'camels_531':
    if GAGE_ID in gage_ids_531:
        basin_idx = list(gage_ids_531).index(GAGE_ID)
    else:
        raise ValueError(
            f"Basin with gage ID {GAGE_ID} not found in the CAMELS 531 dataset.",
        )
else:
    raise ValueError(
        f"Observation data supported: 'camels_671' or 'camels_531'. Got: {config['observations']}",
    )


# 3. Get the data for the chosen basin and plot.
streamflow_pred_basin = pred[:, basin_idx].squeeze()

plot_hydrograph(
    timesteps,
    streamflow_pred_basin,
    resample=RESAMPLE,
    title=f"Hydrograph for Kerrs Creek (Lexington, VA; Gage {GAGE_ID})",
    ylabel='Streamflow (mm/day)',
)

<br>

## 2. Train δHBV 1.1p

After completing [these](#before-running) steps, train δHBV 1.1p with the code block below.

**Note**
- The settings defined in the config [config_dhbv_1_1p.yaml](./../conf/config_dhbv_1_1p.yaml) are set to replicate benchmark performance.
- For model training, set `mode: train` in the config, or modify after config dict has been created (see below).
- `./output/` directory will be generated to store experiment and model files. This location can be adjusted by changing the `output_dir` key in your config. 
    - If you have set `model_dir` in your config, model save files will be stored there.
- Default settings with 50 epochs, batch size 100, and training window from 1 October 1999 to 30 September 2008 should use ~3.5GB of VRAM. Expect training times of ~5 hours on Nvidia A100.

In [ ]:
import sys

sys.path.append('../../')

from example import load_config

from dmg import ModelHandler
from dmg.core.utils import (
    import_data_loader,
    import_trainer,
    print_config,
    set_randomseed,
)

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#


# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'train'
print_config(config)

# Set random seed for reproducibility.
set_randomseed(config['seed'])

# 2. Initialize the differentiable dHBV 1.1p model (LSTM + HBV 1.1p) with model handler.
model = ModelHandler(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=True, overwrite=False)


# 4. Initialize trainer to handle model training.
trainer_cls = import_trainer(config['trainer'])
trainer = trainer_cls(
    config,
    model,
    train_dataset=data_loader.train_dataset,
)

# 5. Start model training.
trainer.train()
print(f"Training complete. Model saved to \n{config['output_dir']}")

## 3. Evaluate Model Performance

After completing the training in [Section 2](#2-train-δhbv-11p), or with the trained model provided, test δHBV 1.1p below on the evaluation data.

**Note**
- For model evaluation, set `mode: test` in the config, or modify after config has been created (see below).
- When evaluating provided models, confirm that `test.test_epoch` in the config corresponds the training epochs completed for the model you want to test (e.g., 50 or 100).
- Default settings with batch size 25 and testing window from 1 October 1989 to 30 September 1999 should use ~2.3GB of VRAM. Expect evalutation times of ~45 seconds on Nvidia A100.

### 3.1 Streamflow Simulation

In [ ]:
import sys

sys.path.append('../../')

from example import load_config

from dmg import ModelHandler
from dmg.core.utils import (
    import_data_loader,
    import_trainer,
    print_config,
    set_randomseed,
)

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#


# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'test'
print_config(config)

set_randomseed(config['seed'])

# 2. Initialize the differentiable dHBV 1.1p model (LSTM + HBV 1.1p).
model = ModelHandler(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=True, overwrite=False)

# 4. Initialize trainer to handle model evaluation.
trainer_cls = import_trainer(config['trainer'])
trainer = trainer_cls(
    config,
    model,
    eval_dataset=data_loader.eval_dataset,
    verbose=True,
)

# 5. Start testing the model.
print('Evaluating model...')
trainer.evaluate()
print(f"Metrics and predictions saved to \n{config['output_dir']}")

### 3.2 Visualize Trained Model Performance

Once the model has been evaluated, `./output/sim/` will be populated with...

1. All model outputs (fluxes, states), including the target variable, *streamflow* (`streamflow.npy`),

2. `streamflow_obs.npy`, streamflow observation data for comparison against model predictions.

Metrics will appear separately in `./output/`:

1. `metrics.json`, containing evaluation metrics accross the test time range for every gage in the dataset,

2. `metrics_agg.json`, containing evaluation metric statistics across all sites (mean, median, standard deviation).

We can use these outputs to visualize δHBV 1.1p's performance with a

1. Cumulative distribution function (CDF) plot, 

2. CONUS map of gage locations and metric (e.g., NSE) performance.

<br>

But first, let's first check the (basin-)aggregated metrics for NSE, KGE, bias, RMSE, and, for both high/low flow regimes, RMSE and absolute percent bias...

In [ ]:
import os

from dmg.core.data import load_json
from dmg.core.post import print_metrics

print(f"Evaluation output files saved to: {config['output_dir']} \n")

# 1. Load the basin-aggregated evaluation results.
metrics_path = os.path.join(config['output_dir'], 'metrics_agg.json')
metrics = load_json(metrics_path)
print(f"Available metrics: {metrics.keys()} \n")

# 2. Print the evaluation results.
metric_names = [
    # Choose metrics to show.
    'nse',
    'kge',
    'bias',
    'rmse',
    'rmse_low',
    'rmse_high',
    'flv_abs',
    'fhv_abs',
]
print_metrics(metrics, metric_names, mode='median', precision=3)

### 3.3 CDF Plot

The cumulative distribution function (CDF) plot tells us what percentage (CDF on the y-axis) of basins performed at least better than a given metric on the evaluation data.

An example is given below for NSE, but you can change to your preferred metric (see the output from the previous cell), but note some may require changing *xbounds* in `plot_cdf()`.

In [ ]:
from dmg.core.post import plot_cdf

# ------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'
# ------------------------------------------#


# 1. Load the evaluation metrics.
metrics_path = os.path.join(config['output_dir'], 'metrics.json')
metrics = load_json(metrics_path)

# 2. Plot the CDF for NSE.
plot_cdf(
    metrics=[metrics],
    metric_names=[METRIC],
    model_labels=['δHBV 1.1p'],
    title="CDF of NSE for δHBV 1.1p",
    xlabel=METRIC.capitalize(),
    figsize=(8, 6),
    xbounds=(0, 1),
    ybounds=(0, 1),
    show_arrow=True,
)

### 3.4 Spatial Plot

This plot shows the locations of each basin in the evaluation data, color-coded by performance on a metric. Here we give a plot for NSE, but as before, this can be changed to your preference. (See above; for metrics not valued between 0 and 1, you will need to set `dynamic_colorbar=True` in `geoplot_single_metric` to ensure proper coding.)

> Notes
>
> 1. You will need to add paths to the CAMELS shapefile, gage IDs, and 531-gage subset which can be found in the [CAMELS download](#before-running).
>
> 2. To use `geoplot_single_metric`, make sure to install dMG with geo plotting dependencies like `uv pip install ./generic_deltamodel[geo]`.

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

from dmg.core.data import txt_to_array
from dmg.core.post import geoplot_single_metric

# ------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'

# Set the paths to the gage id lists and shapefiles...
GAGE_ID_PATH = config['observations']['gage_info']  # ./gage_id.npy
GAGE_ID_531_PATH = config['observations']['subset_path']  # ./531sub_id.txt
SHAPEFILE_PATH = './your/path/to/camels/loc/camels671.shp'
# ------------------------------------------#


# 1. Load gage ids + basin shapefile with geocoordinates (lat, long) for every gage.
gage_ids = np.load(GAGE_ID_PATH, allow_pickle=True)
gage_ids_531 = txt_to_array(GAGE_ID_531_PATH)
coords = gpd.read_file(SHAPEFILE_PATH)

# 2. Format geocoords for 531- and 671-basin CAMELS sets.
coords_531 = coords[coords['gage_id'].isin(list(gage_ids_531))].copy()

coords['gage_id'] = pd.Categorical(
    coords['gage_id'],
    categories=list(gage_ids),
    ordered=True,
)
coords_531['gage_id'] = pd.Categorical(
    coords_531['gage_id'],
    categories=list(gage_ids_531),
    ordered=True,
)

coords = coords.sort_values('gage_id')  # Sort to match order of metrics.
basin_coords_531 = coords_531.sort_values('gage_id')

# 3. Load the evaluation metrics.
metrics_path = os.path.join(config['output_dir'], 'metrics.json')
metrics = load_json(metrics_path)

# 4. Add the evaluation metrics to the basin shapefile.
if config['observations']['name'] == 'camels_671':
    coords[METRIC] = metrics[METRIC]
    full_data = coords
elif config['observations']['name'] == 'camels_531':
    coords_531[METRIC] = metrics[METRIC]
    full_data = coords_531
else:
    raise ValueError(
        f"Observation data supported: 'camels_671' or 'camels_531'. Got: {config['observations']}",
    )

# 5. Plot the evaluation results spatially.
geoplot_single_metric(
    full_data,
    METRIC,
    rf"Spatial Map of {METRIC.upper()} for δHBV 1.1p on CAMELS "
    f"{config['observations']['name'].split('_')[-1]}",
    dynamic_colorbar=False,
)